In [1]:
import logging
import os

#import wiki_old as w # old wiki
import wiki as w 
 
#from gensim.models import word2vec # for orignal w2v
from localgensim.gensim2.models import word2vec 




#from gensim.models.fasttext import FastText
#from gensim.models.word2vec import Word2Vec # not in use
#from localgensim.gensim2.models.word2vec import Word2Vec # not in use

from tqdm import tqdm

WIKIXML = '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream.xml.bz2'
#WIKIXML = '/home/manni/data/wiki/enwiki-20211120-pages-articles-multistream1.xml-p1p41242.bz2'

In [2]:
print(w.__file__)
print(word2vec.__file__)

/home/manni/ner-s2s/word_embedding/wiki.py
/home/manni/ner-s2s/word_embedding/localgensim/gensim2/models/word2vec.py


In [3]:
import sys
sys.path.append("../../imports/")
import saver as sv

In [4]:
logging.basicConfig(format='[%(asctime)s] %(message)s', level=logging.INFO)
os.makedirs('data/', exist_ok=True)

# Test corpus

In [ ]:
from gensim.test.utils import datapath
from gensim import utils

class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = datapath('lee_background.cor')
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [ ]:
sentences = MyCorpus()

In [ ]:
logging.info('Training model %s', 'word2vec')
model = Word2Vec(sentences, window=5, sg=1, hs=0, negative=10, size=300, sample=0, 
                 workers=1, iter=1, min_count=1)

logging.info('Training done.')

In [ ]:
sentences = w.WikiSentences(WIKIXML, 'en')

# Conll corpus

In [ ]:
sentences = sv.load("conll_corpus")

In [ ]:
model = word2vec.Word2Vec(sentences, window=5, sg=1, hs=0, negative=5, size=300, sample=0, workers=1, iter=1, min_count=1)

In [ ]:
#emb_file = '/mnt/nfs/resdata0/manni/wiki/conll_spx2g.txt'
emb_file = '/mnt/nfs/resdata0/manni/wiki/conll_w2v.txt'

In [ ]:
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(model.wv.vocab), 300))
    for word in tqdm(model.wv.vocab):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

# skip if sentence made

In [5]:
# loc = 'num'|'lr'|'ent'
# pos = True|False
# download latest wiki dump
#w.download_wiki_dump('en', WIKIXML)

# parse wiki dump
wiki_sentences = w.WikiSentences(WIKIXML, 'en',lower=True) # Orignal
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='EM',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='DEP',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='UNS',lower=True,pos=False,loc=False)
#wiki_sentences = w.WikiSentences(WIKIXML, 'en',tokenizer_func='UNSEM',lower=True,pos=False,loc=False)

[2022-09-12 04:07:44,221] Parsing wiki corpus Altered...
[2022-09-12 04:07:52,104] adding document #0 to Dictionary(0 unique tokens: [])
[2022-09-12 04:08:48,790] adding document #10000 to Dictionary(416518 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:09:37,922] adding document #20000 to Dictionary(590924 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:10:20,896] adding document #30000 to Dictionary(733312 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:10:59,818] adding document #40000 to Dictionary(856740 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:11:31,650] adding document #50000 to Dictionary(934779 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:11:50,884] adding document #60000 to Dictionary(955808 unique tokens: ['ability', '

[2022-09-12 04:26:11,431] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:26:11,475] adding document #330000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:26:39,368] discarding 41986 tokens: [('melvega', 1), ('mpkiis', 1), ('ptvclassic', 1), ('tttgold', 1), ('uzmagic', 1), ('wrinklys', 1), ('claireaux', 1), ('esnol', 1), ('nsarkozy', 1), ('vendasi', 1)]...
[2022-09-12 04:26:39,372] keeping 2000000 tokens which were in no less than 0 and no more than 340000 (=100.0%) documents
[2022-09-12 04:26:43,804] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:26:43,869] adding document #340000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:27:10,469] discarding 40748 tokens

[2022-09-12 04:32:44,906] keeping 2000000 tokens which were in no less than 0 and no more than 460000 (=100.0%) documents
[2022-09-12 04:32:49,304] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:32:49,373] adding document #460000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:33:14,455] discarding 46012 tokens: [('arekat', 1), ('erikat', 1), ('neameh', 1), ('ʻrēqāt', 1), ('ʻurayqāt', 1), ('ṣāʼib', 1), ('spikily', 1), ('mammalology', 1), ('ornithognomon', 1), ('bakamatsu', 1)]...
[2022-09-12 04:33:14,457] keeping 2000000 tokens which were in no less than 0 and no more than 470000 (=100.0%) documents
[2022-09-12 04:33:18,822] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:33:18,893] adding document #470000 to Dictionary(2000000 unique 

[2022-09-12 04:38:40,257] adding document #580000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:39:04,522] discarding 41323 tokens: [('qinguangwang', 1), ('songdiwang', 1), ('taishanwang', 1), ('wuguanwang', 1), ('zhuanlunwang', 1), ('五官王', 1), ('十殿阎罗王', 1), ('卞城王', 1), ('宋帝王', 1), ('平等王', 1)]...
[2022-09-12 04:39:04,524] keeping 2000000 tokens which were in no less than 0 and no more than 590000 (=100.0%) documents
[2022-09-12 04:39:09,283] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:39:09,355] adding document #590000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:39:34,027] discarding 39922 tokens: [('noaltris', 1), ('nvingă', 1), ('nţelepţeşce', 1), ('obowiązkom', 1), ('obsije', 1), ('obvari', 1), ('ochotností', 1), ('ochroń', 1), ('oferind', 1)

[2022-09-12 04:44:30,859] keeping 2000000 tokens which were in no less than 0 and no more than 710000 (=100.0%) documents
[2022-09-12 04:44:34,038] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:44:34,089] adding document #710000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:45:00,663] discarding 37586 tokens: [('akbn', 1), ('philidas', 1), ('titanine', 1), ('bulgarianmilitary', 1), ('bouchareine', 1), ('d_fortify_source', 1), ('frasunek', 1), ('lamagra', 1), ('tymm', 1), ('vararg', 1)]...
[2022-09-12 04:45:00,665] keeping 2000000 tokens which were in no less than 0 and no more than 720000 (=100.0%) documents
[2022-09-12 04:45:05,168] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 04:45:05,244] adding document #720000 to Dictionary(200

...

[2022-09-12 07:20:44,470] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:20:44,539] adding document #4090000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:21:07,962] discarding 33321 tokens: [('gerymbas', 1), ('oarthus', 1), ('oryithus', 1), ('aellope', 1), ('aellō', 1), ('aellōpē', 1), ('nikothoē', 1), ('podargē', 1), ('podarkē', 1), ('αελλώπη', 1)]...
[2022-09-12 07:21:07,964] keeping 2000000 tokens which were in no less than 0 and no more than 4100000 (=100.0%) documents
[2022-09-12 07:21:11,778] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:21:11,879] adding document #4100000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:21:34,969] discarding 38442 tokens

[2022-09-12 07:26:58,375] discarding 34852 tokens: [('vinodsingh', 1), ('akademikas', 1), ('beliniche', 1), ('esbf', 1), ('mtsv', 1), ('neerbeek', 1), ('piotrkovia', 1), ('sarabon', 1), ('scbt', 1), ('tecnisa', 1)]...
[2022-09-12 07:26:58,377] keeping 2000000 tokens which were in no less than 0 and no more than 4220000 (=100.0%) documents
[2022-09-12 07:27:03,237] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:27:03,339] adding document #4220000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:27:31,545] discarding 26185 tokens: [('pūce', 1), ('atsgpmgwlpvfyrf', 1), ('conorfamide', 1), ('conorfamides', 1), ('farps', 1), ('fmrfa', 1), ('edelgasse', 1), ('jaskulsky', 1), ('witiska', 1), ('clericofascist', 1)]...
[2022-09-12 07:27:31,546] keeping 2000000 tokens which were in no less than 0 and no more than 4230000 (=100.0%) docu

[2022-09-12 07:33:17,566] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:33:17,635] adding document #4340000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:33:42,653] discarding 31583 tokens: [('takkeja', 1), ('taloss', 1), ('tuemme', 1), ('ulkont', 1), ('voisitko', 1), ('voisitsä', 1), ('ähvä', 1), ('ähävä', 1), ('öylen', 1), ('adverbi', 1)]...
[2022-09-12 07:33:42,655] keeping 2000000 tokens which were in no less than 0 and no more than 4350000 (=100.0%) documents
[2022-09-12 07:33:47,580] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:33:47,684] adding document #4350000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:34:12,167] discarding 27685 tokens: [('yncm

[2022-09-12 07:39:54,515] discarding 31226 tokens: [('daluk', 1), ('gadumba', 1), ('hvii', 1), ('lanuva', 1), ('perume', 1), ('rathugala', 1), ('stegeborn', 1), ('talagoya', 1), ('thissahamy', 1), ('tisahamy', 1)]...
[2022-09-12 07:39:54,517] keeping 2000000 tokens which were in no less than 0 and no more than 4470000 (=100.0%) documents
[2022-09-12 07:39:59,396] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:39:59,497] adding document #4470000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:40:23,315] discarding 29409 tokens: [('autonamiste', 1), ('a_stream_on_efate', 1), ('endehipa', 1), ('hamlinson', 1), ('südpazifik', 1), ('vunic', 1), ('forari', 1), ('mardiganyan', 1), ('pswps', 1), ('groterol', 1)]...
[2022-09-12 07:40:23,316] keeping 2000000 tokens which were in no less than 0 and no more than 4480000 (=100.0%) docume

[2022-09-12 07:46:00,182] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:46:00,286] adding document #4590000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:46:26,107] discarding 34129 tokens: [('nanotechnique', 1), ('minskytron', 1), ('riskbank', 1), ('omprises', 1), ('ashvattama', 1), ('dhṛtarāṣtra', 1), ('diparvan', 1), ('naimiśa', 1), ('pañcavimśa', 1), ('pāñcarātrin', 1)]...
[2022-09-12 07:46:26,109] keeping 2000000 tokens which were in no less than 0 and no more than 4600000 (=100.0%) documents
[2022-09-12 07:46:29,701] resulting dictionary: Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:46:29,771] adding document #4600000 to Dictionary(2000000 unique tokens: ['ability', 'abolition', 'abstentionism', 'academic', 'accept']...)
[2022-09-12 07:46:56,817

In [6]:
for sent in wiki_sentences:
    print(sent[:100])
    break

['anarchism', 'political', 'philosophy', 'movement', 'sceptical', 'authority#4', 'reject', 'involuntary', 'coercive', 'form#11', 'hierarchy', 'anarchism', 'call#24', 'abolition', 'state', 'hold#15', 'unnecessary', 'undesirable', 'harmful', 'historically', 'left#7', 'wing#7', 'movement#3', 'placed', 'farthest', 'left', 'political', 'spectrum', 'usually', 'described', 'alongside', 'libertarian', 'marxism', 'libertarian', 'wing#7', 'libertarian', 'socialism', 'socialist', 'movement', 'ha', 'strong', 'historical', 'association', 'anti', 'capitalism', 'socialism', 'human', 'lived', 'society#1', 'without', 'formal', 'hierarchy#1', 'long', 'establishment#2', 'formal', 'state#3', 'realm', 'empire#1', 'rise#25', 'organised', 'hierarchical', 'body', 'scepticism', 'toward', 'authority#4', 'also', 'rose#18', 'th', 'century', 'self', 'conscious', 'political', 'movement#3', 'emerged', 'latter', 'half#1', 'th', 'first#11', 'decade', 'th', 'century', 'anarchist', 'movement', 'flourished', 'part#8', 'w

In [7]:
#sv.save(wiki_sentences,"wiki_sentences_pos_sample")
#sv.save(wiki_sentences,"wiki_sentences_pos")
#sv.save(wiki_sentences,"wiki_sentences_dep")
#sv.save(wiki_sentences,"wiki_sentences_sp")
#sv.save(wiki_sentences,"wiki_sentences_sp_loc")
#sv.save(wiki_sentences,"wiki_sentences_sp_ent")
#sv.save(wiki_sentences,"wiki_sentences_sp_ent_sample")
#sv.save(wiki_sentences,"wiki_sentences") # orignal
#sv.save(wiki_sentences,"wiki_sentences_dep2")
#sv.save(wiki_sentences,"wiki_sentences_uns")
#sv.save(wiki_sentences,"wiki_sentences_unsem")
#sv.save(wiki_sentences,"wiki_sentences_em")
sv.save(wiki_sentences,"wiki_sentences_wnet")

Saved the data


# Phrase mining

In [ ]:
from gensim.test.utils import datapath
from gensim.models.phrases import Phrases

In [ ]:
phrases = Phrases(sentences, min_count=100, threshold=1)
frozen_phrases = phrases.freeze()

In [ ]:
sv.save(phrases,"gensim_phrases")

# Train procedure

In [8]:
#sentences = sv.load("wiki_sentences_no")
#temp_sens are cased!!
#sentences = sv.load("temp_sens")
 
#sentences = sv.load("wiki_sentences") #Normal sentences using wiki_old.py

#Wiki_Sentences_SP are cased
#sentences = sv.load("Wiki_Sentences_SP")

#sentences = sv.load("wiki_sentences_sp_loc") #New
#sentences = sv.load("wiki_sentences_sp") #New

#sentences = sv.load("wiki_sentences_pos") # not to be used
#sentences = sv.load("Wiki_sentences_pos_sample")

#sentences = sv.load("wiki_sentences_sp_ent") # New
#sentences = sv.load("wiki_sentences_sp_ent_sample") # New

#sentences = sv.load("wiki_sentences_dep") #New
#sentences = sv.load("wiki_sentences_dep2") #New

#wiki english sample Cased 
#sentences = sv.load("Wiki_sentences_sp_sample")
#sentences = sv.load("wiki_sentences_uns") #New
#sentences = sv.load("wiki_sentences_unsem") #New
#sentences = sv.load("wiki_sentences_em") #New
sentences = sv.load("wiki_sentences_wnet") #New

In [ ]:
#sentences = wiki_sentences
print("Minimum length of token:",sentences.wiki.token_min_len)

In [9]:
logging.info('Training model %s', 'wordnet')
model = word2vec.Word2Vec(sentences, window=10, sg=1, hs=0, negative=5, size=300, workers=40, iter=5)
logging.info('Training done.')

[2022-09-12 09:57:00,595] Training model wordnet
[2022-09-12 09:57:00,674] collecting all words and their counts
[2022-09-12 09:57:12,969] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[2022-09-12 09:59:49,872] PROGRESS: at sentence #40000, processed 56384702 words, keeping 856740 word types
[2022-09-12 10:00:14,854] PROGRESS: at sentence #50000, processed 65366350 words, keeping 934779 word types
[2022-09-12 10:00:30,295] PROGRESS: at sentence #60000, processed 70630249 words, keeping 955808 word types
[2022-09-12 10:00:45,143] PROGRESS: at sentence #70000, processed 75423773 words, keeping 974931 word types
[2022-09-12 10:00:58,905] PROGRESS: at sentence #80000, processed 79662425 words, keeping 990589 word types
[2022-09-12 10:01:28,628] PROGRESS: at sentence #90000, processed 90797618 words, keeping 1082983 word types
[2022-09-12 10:01:59,697] PROGRESS: at sentence #100000, processed 102457372 words, keeping 1190222 word types
[2022-09-12 10:02:28,827] PROGRESS:

[2022-09-12 10:24:45,720] PROGRESS: at sentence #770000, processed 527113515 words, keeping 3867968 word types
[2022-09-12 10:25:01,922] PROGRESS: at sentence #780000, processed 531734109 words, keeping 3894613 word types
[2022-09-12 10:25:19,755] PROGRESS: at sentence #790000, processed 536364962 words, keeping 3918372 word types
[2022-09-12 10:25:36,897] PROGRESS: at sentence #800000, processed 540844557 words, keeping 3941040 word types
[2022-09-12 10:25:53,594] PROGRESS: at sentence #810000, processed 545399062 words, keeping 3965949 word types
[2022-09-12 10:26:09,798] PROGRESS: at sentence #820000, processed 549883799 words, keeping 3989244 word types
[2022-09-12 10:26:26,961] PROGRESS: at sentence #830000, processed 554487304 words, keeping 4013161 word types
[2022-09-12 10:26:44,329] PROGRESS: at sentence #840000, processed 559004663 words, keeping 4036785 word types
[2022-09-12 10:26:59,832] PROGRESS: at sentence #850000, processed 563448366 words, keeping 4059165 word types
[

[2022-09-12 10:44:52,658] PROGRESS: at sentence #1510000, processed 824594635 words, keeping 5375855 word types
[2022-09-12 10:45:07,674] PROGRESS: at sentence #1520000, processed 827725201 words, keeping 5389953 word types
[2022-09-12 10:45:22,052] PROGRESS: at sentence #1530000, processed 830784851 words, keeping 5402293 word types
[2022-09-12 10:45:39,529] PROGRESS: at sentence #1540000, processed 834250882 words, keeping 5417749 word types
[2022-09-12 10:45:55,707] PROGRESS: at sentence #1550000, processed 837745141 words, keeping 5436575 word types
[2022-09-12 10:46:11,968] PROGRESS: at sentence #1560000, processed 841267617 words, keeping 5456356 word types
[2022-09-12 10:46:29,095] PROGRESS: at sentence #1570000, processed 844942723 words, keeping 5476552 word types
[2022-09-12 10:46:47,265] PROGRESS: at sentence #1580000, processed 848391547 words, keeping 5492185 word types
[2022-09-12 10:47:04,831] PROGRESS: at sentence #1590000, processed 851929512 words, keeping 5509966 wor

[2022-09-12 11:06:14,336] PROGRESS: at sentence #2240000, processed 1077121203 words, keeping 6568724 word types
[2022-09-12 11:06:30,992] PROGRESS: at sentence #2250000, processed 1080364265 words, keeping 6592388 word types
[2022-09-12 11:06:49,445] PROGRESS: at sentence #2260000, processed 1083848641 words, keeping 6610708 word types
[2022-09-12 11:07:07,379] PROGRESS: at sentence #2270000, processed 1087233298 words, keeping 6628263 word types
[2022-09-12 11:07:24,498] PROGRESS: at sentence #2280000, processed 1090619789 words, keeping 6644115 word types
[2022-09-12 11:07:41,829] PROGRESS: at sentence #2290000, processed 1094074855 words, keeping 6659501 word types
[2022-09-12 11:08:00,313] PROGRESS: at sentence #2300000, processed 1097684389 words, keeping 6675039 word types
[2022-09-12 11:08:17,522] PROGRESS: at sentence #2310000, processed 1101218844 words, keeping 6690457 word types
[2022-09-12 11:08:33,650] PROGRESS: at sentence #2320000, processed 1104561342 words, keeping 67

[2022-09-12 11:28:16,051] PROGRESS: at sentence #2970000, processed 1325712650 words, keeping 7731016 word types
[2022-09-12 11:28:35,169] PROGRESS: at sentence #2980000, processed 1329122933 words, keeping 7745588 word types
[2022-09-12 11:28:53,781] PROGRESS: at sentence #2990000, processed 1332525926 words, keeping 7757426 word types
[2022-09-12 11:29:13,339] PROGRESS: at sentence #3000000, processed 1335866692 words, keeping 7771491 word types
[2022-09-12 11:29:31,211] PROGRESS: at sentence #3010000, processed 1339011451 words, keeping 7785068 word types
[2022-09-12 11:29:49,885] PROGRESS: at sentence #3020000, processed 1342313948 words, keeping 7800396 word types
[2022-09-12 11:30:08,564] PROGRESS: at sentence #3030000, processed 1345488024 words, keeping 7812176 word types
[2022-09-12 11:30:27,331] PROGRESS: at sentence #3040000, processed 1348852392 words, keeping 7830617 word types
[2022-09-12 11:30:45,192] PROGRESS: at sentence #3050000, processed 1352041233 words, keeping 78

...

[2022-09-12 22:48:04,593] EPOCHs No. 3 - PROGRESS: at 63.00% examples, 143726 words/s, in_qsize 0, out_qsize 0
[2022-09-12 22:50:10,408] EPOCHs No. 3 - PROGRESS: at 64.00% examples, 143411 words/s, in_qsize 0, out_qsize 1
[2022-09-12 22:52:18,688] EPOCHs No. 3 - PROGRESS: at 65.00% examples, 143087 words/s, in_qsize 0, out_qsize 0
[2022-09-12 22:54:37,687] EPOCHs No. 3 - PROGRESS: at 66.00% examples, 142693 words/s, in_qsize 0, out_qsize 0
[2022-09-12 22:56:49,699] EPOCHs No. 3 - PROGRESS: at 67.00% examples, 142375 words/s, in_qsize 0, out_qsize 0
[2022-09-12 22:59:02,583] EPOCHs No. 3 - PROGRESS: at 68.00% examples, 142116 words/s, in_qsize 0, out_qsize 0
[2022-09-12 23:01:09,314] EPOCHs No. 3 - PROGRESS: at 69.00% examples, 141850 words/s, in_qsize 0, out_qsize 0
[2022-09-12 23:03:16,920] EPOCHs No. 3 - PROGRESS: at 70.00% examples, 141611 words/s, in_qsize 0, out_qsize 0
[2022-09-12 23:05:20,181] EPOCHs No. 3 - PROGRESS: at 71.00% examples, 141399 words/s, in_qsize 0, out_qsize 0
[

[2022-09-13 00:27:29,937] EPOCHs No. 4 - PROGRESS: at 4.00% examples, 155626 words/s, in_qsize 0, out_qsize 0
[2022-09-13 00:31:16,928] EPOCHs No. 4 - PROGRESS: at 5.00% examples, 156587 words/s, in_qsize 0, out_qsize 0
[2022-09-13 00:34:50,779] EPOCHs No. 4 - PROGRESS: at 6.00% examples, 156884 words/s, in_qsize 0, out_qsize 0
[2022-09-13 00:38:02,892] EPOCHs No. 4 - PROGRESS: at 7.00% examples, 157360 words/s, in_qsize 0, out_qsize 0
[2022-09-13 00:41:06,493] EPOCHs No. 4 - PROGRESS: at 8.00% examples, 157653 words/s, in_qsize 0, out_qsize 1
[2022-09-13 00:44:06,908] EPOCHs No. 4 - PROGRESS: at 9.00% examples, 157587 words/s, in_qsize 0, out_qsize 0
[2022-09-13 00:46:56,226] EPOCHs No. 4 - PROGRESS: at 10.00% examples, 157723 words/s, in_qsize 0, out_qsize 3
[2022-09-13 00:49:45,051] EPOCHs No. 4 - PROGRESS: at 11.00% examples, 157692 words/s, in_qsize 0, out_qsize 0
[2022-09-13 00:52:23,942] EPOCHs No. 4 - PROGRESS: at 12.00% examples, 157673 words/s, in_qsize 6, out_qsize 0
[2022-0

[2022-09-13 03:11:07,361] EPOCHs No. 4 - PROGRESS: at 78.00% examples, 139580 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:13:12,064] EPOCHs No. 4 - PROGRESS: at 79.00% examples, 139371 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:15:20,934] EPOCHs No. 4 - PROGRESS: at 80.00% examples, 139067 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:17:20,008] EPOCHs No. 4 - PROGRESS: at 81.00% examples, 138870 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:19:22,396] EPOCHs No. 4 - PROGRESS: at 82.00% examples, 138624 words/s, in_qsize 0, out_qsize 1
[2022-09-13 03:21:28,776] EPOCHs No. 4 - PROGRESS: at 83.00% examples, 138383 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:23:34,136] EPOCHs No. 4 - PROGRESS: at 84.00% examples, 138136 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:25:40,186] EPOCHs No. 4 - PROGRESS: at 85.00% examples, 137893 words/s, in_qsize 0, out_qsize 0
[2022-09-13 03:28:02,522] EPOCHs No. 4 - PROGRESS: at 86.00% examples, 137470 words/s, in_qsize 0, out_qsize 0
[

[2022-09-13 05:00:45,806] EPOCHs No. 5 - PROGRESS: at 19.00% examples, 156347 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:02:59,997] EPOCHs No. 5 - PROGRESS: at 20.00% examples, 156146 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:05:07,472] EPOCHs No. 5 - PROGRESS: at 21.00% examples, 155995 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:07:12,795] EPOCHs No. 5 - PROGRESS: at 22.00% examples, 155802 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:09:13,753] EPOCHs No. 5 - PROGRESS: at 23.00% examples, 155632 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:11:19,812] EPOCHs No. 5 - PROGRESS: at 24.00% examples, 155371 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:13:18,385] EPOCHs No. 5 - PROGRESS: at 25.00% examples, 155248 words/s, in_qsize 0, out_qsize 2
[2022-09-13 05:15:21,508] EPOCHs No. 5 - PROGRESS: at 26.00% examples, 155014 words/s, in_qsize 0, out_qsize 0
[2022-09-13 05:17:27,906] EPOCHs No. 5 - PROGRESS: at 27.00% examples, 154726 words/s, in_qsize 0, out_qsize 1
[

[2022-09-13 07:36:07,145] EPOCHs No. 5 - PROGRESS: at 93.00% examples, 134954 words/s, in_qsize 0, out_qsize 0
[2022-09-13 07:38:33,159] EPOCHs No. 5 - PROGRESS: at 94.00% examples, 134607 words/s, in_qsize 0, out_qsize 0
[2022-09-13 07:40:53,881] EPOCHs No. 5 - PROGRESS: at 95.00% examples, 134285 words/s, in_qsize 0, out_qsize 1
[2022-09-13 07:43:04,135] EPOCHs No. 5 - PROGRESS: at 96.00% examples, 134034 words/s, in_qsize 0, out_qsize 0
[2022-09-13 07:45:17,748] EPOCHs No. 5 - PROGRESS: at 97.00% examples, 133786 words/s, in_qsize 0, out_qsize 1
[2022-09-13 07:47:33,531] EPOCHs No. 5 - PROGRESS: at 98.00% examples, 133497 words/s, in_qsize 0, out_qsize 0
[2022-09-13 07:49:45,850] EPOCHs No. 5 - PROGRESS: at 99.00% examples, 133186 words/s, in_qsize 0, out_qsize 0
[2022-09-13 07:51:49,937] finished iterating over Wikipedia corpus of 4730463 documents with 1888752251 positions (total 21639335 articles, 1969858045 positions before pruning articles shorter than 50 words)
[2022-09-13 07:

In [10]:
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_filtered_sample.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_w2v_mc1_epoch5_300_sample.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2R_mc1_epoch5_300_filtered.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2w2v_mc1_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_con1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc1_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_neg10.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2_mc100_epoch5_300_neg10_w3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2S_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_SPX2B_mc100_epoch5_300_sub3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2TB_mc100_epoch5_300_LR.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2POS_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2DEP_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2LRM3_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2LOC_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_reversed.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx_mc100_epoch5_300_loc.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_w2v_mc100_epoch5_300_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_pos.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx_mc100_epoch5_300_ent_w10.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_dep2_w1.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_ent_static_w3.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_uns.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_unsem.txt'
#emb_file = '/home/manni/embs/en_wiki_spx_mc100_epoch5_300_em.txt'
#emb_file = '/mnt/nfs/resdata0/manni/wiki/en_wiki_spx2_mc100_epoch5_300_uns_w1.txt'
emb_file = '/home/manni/embs/en_wiki_wnet_epoch5_300.txt'

In [11]:
vocab = model.wv.vocab

In [12]:
len(vocab)

2487755

In [13]:
vocab.pop('[', None)
vocab.pop(']', None)
len(vocab)

2487755

In [14]:
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vocab), 300))
    for word in tqdm(vocab, position=0):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

[2022-09-13 07:51:51,659] Save trained word vectors
 37%|████████████████████████████████████████████████████████▏                                                                                             | 932086/2487755 [05:22<08:30, 3047.34it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2487755/2487755 [14:14<00:00, 2912.73it/s]
[2022-09-13 08:06:08,453] Done
